Chap5_CNN
==========
간단한 합성곱 신경망 구현
------------------------------------

In [3]:
import torch
import torch.nn as nn                        # 신경망
import torch.optim as optim                  # 최적화
import torch.nn.init as init                 # 텐서에 초기값을 주기위한 험수
import torchvision.datasets as dset          # 모듈 데이터를 읽어옴
import torchvision.transforms as transforms  # 불러온 이미지를 변환
from torch.utils.data import DataLoader      # DataLoader모듈

from tqdm.notebook import tqdm

In [4]:
# Hyperparameter
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

## MNIST 데이터셋을 사용.
### dset.MNIST (28 by 28의 이미지)
dset.MNIST(데이터경로, train=True(학습 데이터 불러옴), train=False(테스트 데이터 불러옴),  
transform & target_transform(각각 데이터와 라벨에 대한 변형을 의미.)  
transform.ToTensor()는 이미지 데이터를 파이토치 텐서로 변환하겠다는 의미.  
download(현재경로에 MNIST데이터가 없을 경우에 다운로드를 하겠다는 의미.)
* * *
### torch.utils.data.DataLoader
batch_size: dset.MNIST를 통해 정리된 데이터를 batch_size 개수만큼 묶는다는 의미  
shuffle: 셔플여부(데이터를 데이터로더에서 섞어서 사용을 하겠는지 여부)  
num_workers: 데이터를 묶을 때 사용할 프로세스 개수  
drop_last: 묶고 남은 데이터를 버릴지에 대한 여부

In [5]:
mnist_train = dset.MNIST("./", train=True, transform = transforms.ToTensor(),
                        target_transform=None, download=True)
mnist_test  = dset.MNIST("./", train=False, transform = transforms.ToTensor(),
                        target_transform = None, download=True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size = batch_size,
                                          shuffle = True, num_workers = 2, drop_last = True)
test_loader  = torch.utils.data.DataLoader(mnist_test,  batch_size = batch_size,
                                          shuffle = False,num_workers = 2, drop_last = True)


## Neural Net CNN
* * *
Conv2d의 인수 (입력채널, 출력채널, kernel_size, stride, padding 등)  
super는 CNN클래스의 부모 클래스인 nn.Module을 초기화 하는 역할을 해줌.  
입력채널에는 RGB 3채널 데이터를 입력으로 받음.  
16은 필터의 갯수  
kernel_size: 5 (임의의 값)
***
Tensor의 형태
--------------------
입력: [batch_size, input_channel,  가로, 세로]  
출력: [batch_size, output_channel, 가로, 세로]  

### 연산식
$$
O=floor(\frac{I-K+2P}{S}+1)
$$
I: 이미지의 크기(입력의 한변의 크기)  
K: 필터(커널)의 크기  
P: 패딩의 크기  
S: 스트라이드 기본값 1  
  
문제: nn.Conv2d(1, 16, 5)를 통과한 이미지의 사이즈?
답: 24 

### Max Pooling
nn.MaxPool2d 인수: kernel_size(풀링 연산의 크기), stride=2 풀링을 하고 2만큼 이동  
해당 연산이후 텐서가 반으로 줄어듬.  
결과: [batch_size, 32,  20, 20] -> [batch_size, 32,  10, 10]

### torch.view  
예를들어 [4,16]의 형태의 텐서를 torch.view(2, -1)를 거치면, [2,32]의 텐서를 얻음.

### fc_layer



In [13]:
"""
make class CNN
"""
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),# [batch_size, 32,  10, 10]
            nn.Conv2d(32,64,5),
            nn.ReLU(),# [batch_size, 64,  6, 6]
            nn.MaxPool2d(2,2) # [batch_size, 64,  3, 3]
        )    
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )
        
    def forward(self, x_input):
        out = self.layer(x_input)
        out = out.view(batch_size, -1) # -1은 -1부분을 알아서 계산하라는 의미
        out = self.fc_layer(out) # [batch_size, 10]

        return out

In [14]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = CNN().to(device)
loss_f = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
loss_arr = []
for i in tqdm(range(num_epoch)):
    for j,[image, label] in enumerate(train_loader):
        x  = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_f(output,y_)
        loss.backward()
        optimizer.step()
        
        if j % 1000 == 0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())

tensor(0.0604, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0215, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0121, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0436, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0128, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0157, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0160, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0229, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0785, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0243, device='cuda:0', grad_fn=<NllLossBackward>)



In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        x  = image.to(device)
        y_ = label.to(device)
        
        output = model.forward(x)
        _, output_index = torch.max(output,1)
        
        total += label.size(0)
        correct += (output_index == y_).sum().float()
        
    print("Accuracy of Test Data: {}".format(100*correct/total))